# Importing usefull libraries and scripts

In [20]:
import pickle
import warnings
import requests

warnings.filterwarnings('ignore')

# Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

In [5]:
!pipenv --version

pipenv, version 2022.10.12


# Question 2

* Use Pipenv to install Scikit-Learn version 1.0.2
* What's the first hash for scikit-learn you get in Pipfile.lock?

Note: you should create an empty folder for homework
and do it there. 

> "sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b"

# Question 3

Let's use these models!

* Write a script for loading these models with pickle
* Score this client:

```json
{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}
```

What's the probability that this client will get a credit card? 

* 0.162
* 0.391
* 0.601
* 0.993

If you're getting errors when unpickling the files, check their checksum:

```bash
$ md5sum model1.bin dv.bin
3f57f3ebfdf57a9e1368dcd0f28a4a14  model1.bin
6b7cded86a52af7e81859647fa3a5c2e  dv.bin

In [8]:
PREFIX= "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework"
!wget $PREFIX/model1.bin
!wget $PREFIX/dv.bin

--2022-10-25 12:29:37--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889 [application/octet-stream]
Saving to: ‘model1.bin’

model1.bin          100%[===================>]     889  --.-KB/s    in 0s      

2022-10-25 12:29:38 (20.8 MB/s) - ‘model1.bin’ saved [889/889]

--2022-10-25 12:29:38--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.1

In [9]:
model_loc = "model1.bin"
dv_loc = "dv.bin"

In [14]:
with open(model_loc,'rb') as model:
    model_ = pickle.load(model)
with open(dv_loc,'rb') as dv:
    dv_ = pickle.load(dv)

In [15]:
customer = {
    "reports": 0, 
    "share": 0.001694, 
    "expenditure": 0.12, 
    "owner": "yes"
    }

In [16]:
X = dv_.transform([customer])

In [19]:
round(model_.predict_proba(X)[0,1],3)

0.162

# Question 4

Now let's serve this model as a web service

* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using `requests`:

```python
url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a credit card?

* 0.274
* 0.484
* 0.698
* 0.928

In [36]:
url_1 = 'http://localhost:9696/predict'
url_2 = 'http://localhost:8080/predict'

In [22]:
client = {
    "reports": 0, 
    "share": 0.245, 
    "expenditure": 3.438, 
    "owner": "yes"
    }

In [41]:
response = requests.post(url_1,json=client).json()

In [42]:
round(response['credit_risk_probability'],3)

0.928

# Docker

Install [Docker](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/05-deployment/06-docker.md). We will use it for the next two questions.

For these questions, we prepared a base image: `svizor/zoomcamp-model:3.9.12-slim`. 
You'll need to use it (see Question 5 for an example).

This image is based on `python:3.9.12-slim` and has a logistic regression model 
(a different one) as well a dictionary vectorizer inside. 

This is how the Dockerfile for this image looks like:

```docker 
FROM python:3.9.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
```

We already built it and then pushed it to [`svizor/zoomcamp-model:3.9.12-slim`](https://hub.docker.com/r/svizor/zoomcamp-model).

> **Note**: You don't need to build this docker image, it's just for your reference.

## Question 5

Download the base image `svizor/zoomcamp-model:3.9.12-slim`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 15 Mb
* 125 Mb
* 275 Mb
* 415 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

> 125MB

# Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

```docker
FROM svizor/zoomcamp-model:3.9.12-slim
# add your stuff here
```

Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with Gunicorn 

After that, you can build your docker image.


## Question 6

Let's run your docker container!

After running it, score this client once again:

```python
url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a credit card now?

* 0.289
* 0.502
* 0.769
* 0.972

In [37]:
client_2 = {
    "reports": 0, 
    "share": 0.245, 
    "expenditure": 3.438, 
    "owner": "yes"
    }

In [38]:
response_2 = requests.post(url_2,json=client_2).json()

In [39]:
round(response_2['credit_risk_probability'],3)

0.769